<a href="https://colab.research.google.com/github/jmoraleses/Activity-deploy-model-brain-tumors/blob/main/Deploy_brain_tumors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clasificación de tumores cerebrales (Javier Morales)

In [ ]:
#@title Ejecuta esta celda para ver el despliegue del modelo { vertical-output: true, display-mode: "form" }
from os import times
!pip install -q ipywidgets
from flask import Flask, render_template, request
from google.colab import drive
import requests
import numpy as np
import tensorflow as tf
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import keras
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from matplotlib import pyplot as plt
from keras.utils import load_img, img_to_array
from PIL import Image


# Autenticarse y crear un cliente de Google Drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Id del archivo
file_id = '1pH2Q6T7n-zcR9cgXTXCsi2YMC-UZkSox'

output_label = widgets.Label(value="")
name_file = widgets.Label(value="")
display(output_label)


# Descargar el modelo desde la URL compartida de Google Drive
downloaded = drive.CreateFile({'id': file_id})
output_label.value = "Cargando modelo..."
downloaded.GetContentFile('model.h5')
output_label.value = "Suba la imagen"

# Cargar el modelo previamente entrenado
model = keras.models.load_model('model.h5')

def on_button_clicked(b):
    uploaded = files.upload()
    for name, data in uploaded.items():
        if name != "":
            with open(name, 'wb') as f:
                f.write(data)
                name_file.value = name
                display(predict_button)
                output_label.value = "Archivo subido"
            
def prediction(b):
    output_label.value = "Procesando..."
    img = Image.open(name_file.value)
    img = img.resize((224, 224), Image.ANTIALIAS)
    img_array = np.array(img)
    img_array2 = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array2)
    print(predictions)
    if int(predictions) == 1:
      output_label.value = "Lamento comunicarle que tiene un tumor cerebral"
    else:
      output_label.value = "Enhorabuena no tiene un tumor cerebral"
    

upload_button = widgets.Button(description="Subir imagen")
upload_button.on_click(on_button_clicked)
display(upload_button)
predict_button = widgets.Button(description="Predecir")
predict_button.on_click(prediction)
